In [1]:
import logging
import sys
from pathlib import Path
from typing import List, Set

import numpy as np
import pandas as pd
import pyarrow as pa
import pyarrow.parquet as pq

# --- 核心配置 ---
# 🛡️ 安全第一: 设为 False 以实际执行文件写入
DRYRUN = False

# --- 路径配置 ---
# 输入: 包含预处理后数据的目录
INPUT_ARTIFACTS_DIR = Path("/cwStorage/nodecw_group/jijh/yuanspace_data/artifacts")
# 输出: 将存放 train/ 和 val/ 子目录的基准目录
OUTPUT_SPLIT_DIR = Path("/cwStorage/nodecw_group/jijh/yuanspace_data/dataset_split")

# 定义原始清单文件路径，用于获取 sample_id
ORIGINAL_MANIFEST_FILE = Path("/cwStorage/nodecw_group/jijh/hest_1k/HEST_v1_1_0.csv") 

# 检查输入文件是否存在
assert INPUT_ARTIFACTS_DIR.exists(), f"输入目录不存在: {INPUT_ARTIFACTS_DIR}"
assert ORIGINAL_MANIFEST_FILE.exists(), f"原始清单文件不存在: {ORIGINAL_MANIFEST_FILE}"
for filename in ["nodes.parquet", "edges.parquet", "image_embeds.npy", "text_embeds.npy"]:
    assert (INPUT_ARTIFACTS_DIR / filename).exists(), f"输入文件缺失: {filename}"

# --- 日志配置 ---
logging.basicConfig(
    level=logging.INFO,
    format='%(asctime)s - %(levelname)s - %(message)s',
    stream=sys.stdout
)

In [2]:
def get_validation_sample_ids(manifest_path: Path) -> Set[str]:
    """
    根据预定义的研究项目标题，从原始清单中获取验证集的 sample_id 列表。
    """
    df_manifest = pd.read_csv(manifest_path)
    
    VALIDATION_TITLES = [
        "Spatial deconvolution of HER2-positive breast cancer delineates tumor-associated cell type interactions",
        "Characterization of immune cell populations in the tumor microenvironment of colorectal cancer using high definition spatial profiling"
    ]
    
    validation_ids = df_manifest[df_manifest['dataset_title'].isin(VALIDATION_TITLES)]['id'].unique()
    
    logging.info(f"从清单中确定了 {len(validation_ids)} 个用于验证的独立样本 ID。")
    return set(validation_ids)

def split_and_save_dataset(
    input_dir: Path, 
    output_dir: Path, 
    validation_sample_ids: Set[str]
):
    """
    加载预处理的数据，按 sample_id 拆分为训练/验证集，并保存到新目录。
    """
    logging.info(f"开始从 '{input_dir}' 加载数据...")
    nodes_df = pd.read_parquet(input_dir / "nodes.parquet")
    edges_df = pd.read_parquet(input_dir / "edges.parquet")
    all_img_embeds = np.load(input_dir / "image_embeds.npy")
    all_txt_embeds = np.load(input_dir / "text_embeds.npy")
    logging.info("所有输入文件加载完毕。")

    # 1. 拆分 Nodes 和 Embeddings
    val_nodes_mask = nodes_df['sample_id'].isin(validation_sample_ids)
    
    train_nodes_df = nodes_df[~val_nodes_mask].copy()
    val_nodes_df = nodes_df[val_nodes_mask].copy()
    
    # 使用原始索引来切片 numpy 数组
    train_indices = train_nodes_df.index.to_numpy()
    val_indices = val_nodes_df.index.to_numpy()
    
    train_img_embeds = all_img_embeds[train_indices]
    val_img_embeds = all_img_embeds[val_indices]
    
    train_txt_embeds = all_txt_embeds[train_indices]
    val_txt_embeds = all_txt_embeds[val_indices]
    
    # 2. 拆分 Edges
    train_tile_ids = set(train_nodes_df['tile_id'])
    val_tile_ids = set(val_nodes_df['tile_id'])
    
    train_edges_mask = edges_df['src_tile_id'].isin(train_tile_ids) & edges_df['nbr_tile_id'].isin(train_tile_ids)
    val_edges_mask = edges_df['src_tile_id'].isin(val_tile_ids) & edges_df['nbr_tile_id'].isin(val_tile_ids)
    
    train_edges_df = edges_df[train_edges_mask].copy()
    val_edges_df = edges_df[val_edges_mask].copy()

    # 3. 报告统计数据
    logging.info("--- 拆分统计 ---")
    logging.info(f"训练集: {len(train_nodes_df)} tiles, {len(train_edges_df)} edges")
    logging.info(f"验证集: {len(val_nodes_df)} tiles, {len(val_edges_df)} edges")
    logging.info("--------------------")

    # 4. 安全写入文件
    if not DRYRUN:
        train_dir = output_dir / "train"
        val_dir = output_dir / "val"
        train_dir.mkdir(parents=True, exist_ok=True)
        val_dir.mkdir(parents=True, exist_ok=True)
        
        logging.info(f"正在向 '{train_dir}' 写入训练集文件...")
        pq.write_table(pa.Table.from_pandas(train_nodes_df, preserve_index=False), train_dir / "nodes.parquet")
        pq.write_table(pa.Table.from_pandas(train_edges_df, preserve_index=False), train_dir / "edges.parquet")
        np.save(train_dir / "image_embeds.npy", train_img_embeds)
        np.save(train_dir / "text_embeds.npy", train_txt_embeds)
        
        logging.info(f"正在向 '{val_dir}' 写入验证集文件...")
        pq.write_table(pa.Table.from_pandas(val_nodes_df, preserve_index=False), val_dir / "nodes.parquet")
        pq.write_table(pa.Table.from_pandas(val_edges_df, preserve_index=False), val_dir / "edges.parquet")
        np.save(val_dir / "image_embeds.npy", val_img_embeds)
        np.save(val_dir / "text_embeds.npy", val_txt_embeds)
        
        logging.info("所有文件已成功写入磁盘。")
    else:
        logging.warning("当前为预演模式 (DRYRUN=True)，未执行任何写盘操作。")

In [3]:
def validate_split(output_dir: Path):
    """
    加载拆分后的数据并执行一系列检查以确保其完整性和正确性。
    """
    logging.info("--- 开始验证拆分后的数据集 ---")
    train_dir = output_dir / "train"
    val_dir = output_dir / "val"
    
    if not train_dir.exists() or not val_dir.exists():
        logging.error("验证失败: 输出目录 train/ 或 val/ 不存在。")
        return

    # 加载数据
    train_nodes = pd.read_parquet(train_dir / "nodes.parquet")
    val_nodes = pd.read_parquet(val_dir / "nodes.parquet")
    train_edges = pd.read_parquet(train_dir / "edges.parquet")
    val_edges = pd.read_parquet(val_dir / "edges.parquet")
    train_img_embeds = np.load(train_dir / "image_embeds.npy")
    val_img_embeds = np.load(val_dir / "image_embeds.npy")

    # 检查 1: 数据泄露 (最重要！)
    train_samples = set(train_nodes['sample_id'].unique())
    val_samples = set(val_nodes['sample_id'].unique())
    common_samples = train_samples.intersection(val_samples)
    
    assert not common_samples, f"验证失败：发现数据泄露！以下 sample_id 同时存在于训练集和验证集: {common_samples}"
    logging.info("✅ PASSED: 训练集与验证集之间无样本ID交集 (无数据泄露)。")
    
    # 检查 2: 内部一致性
    assert len(train_nodes) == train_img_embeds.shape[0], "训练集节点数与嵌入数不匹配。"
    assert len(val_nodes) == val_img_embeds.shape[0], "验证集节点数与嵌入数不匹配。"
    logging.info("✅ PASSED: 节点数与嵌入数在各子集中保持一致。")

    # 检查 3: 边引用完整性
    train_tile_ids = set(train_nodes['tile_id'])
    val_tile_ids = set(val_nodes['tile_id'])
    assert train_edges['src_tile_id'].isin(train_tile_ids).all(), "训练集边表包含无效的源节点ID。"
    assert train_edges['nbr_tile_id'].isin(train_tile_ids).all(), "训练集边表包含无效的邻居节点ID。"
    assert val_edges['src_tile_id'].isin(val_tile_ids).all(), "验证集边表包含无效的源节点ID。"
    assert val_edges['nbr_tile_id'].isin(val_tile_ids).all(), "验证集边表包含无效的邻居节点ID。"
    logging.info("✅ PASSED: 边表的引用完整性在各子集中得到保持。")
    
    logging.info("🎉 全部验证成功！数据集已准备就绪。")

In [4]:
# --- 主流程 ---
# 1. 从原始清单中获取权威的验证集 sample_id 列表
validation_sample_ids = get_validation_sample_ids(ORIGINAL_MANIFEST_FILE)



2025-09-13 21:49:58,528 - INFO - 从清单中确定了 44 个用于验证的独立样本 ID。


In [5]:
validation_sample_ids

{'SPA119',
 'SPA120',
 'SPA121',
 'SPA122',
 'SPA123',
 'SPA124',
 'SPA125',
 'SPA126',
 'SPA127',
 'SPA128',
 'SPA129',
 'SPA130',
 'SPA131',
 'SPA132',
 'SPA133',
 'SPA134',
 'SPA135',
 'SPA136',
 'SPA137',
 'SPA138',
 'SPA139',
 'SPA140',
 'SPA141',
 'SPA142',
 'SPA143',
 'SPA144',
 'SPA145',
 'SPA146',
 'SPA147',
 'SPA148',
 'SPA149',
 'SPA150',
 'SPA151',
 'SPA152',
 'SPA153',
 'SPA154',
 'TENX147',
 'TENX148',
 'TENX149',
 'TENX152',
 'TENX153',
 'TENX154',
 'TENX155',
 'TENX156'}

In [6]:
# 2. 执行拆分和保存
split_and_save_dataset(
    input_dir=INPUT_ARTIFACTS_DIR,
    output_dir=OUTPUT_SPLIT_DIR,
    validation_sample_ids=validation_sample_ids
)

# 3. 运行验证脚本
if not DRYRUN:
    validate_split(OUTPUT_SPLIT_DIR)

2025-09-13 21:50:08,787 - INFO - 开始从 '/cwStorage/nodecw_group/jijh/yuanspace_data/artifacts' 加载数据...
2025-09-13 21:50:24,688 - INFO - 所有输入文件加载完毕。
2025-09-13 21:50:28,168 - INFO - --- 拆分统计 ---
2025-09-13 21:50:28,171 - INFO - 训练集: 919173 tiles, 5515038 edges
2025-09-13 21:50:28,172 - INFO - 验证集: 33735 tiles, 202410 edges
2025-09-13 21:50:28,173 - INFO - --------------------
2025-09-13 21:50:28,177 - INFO - 正在向 '/cwStorage/nodecw_group/jijh/yuanspace_data/dataset_split/train' 写入训练集文件...
2025-09-13 21:50:50,925 - INFO - 正在向 '/cwStorage/nodecw_group/jijh/yuanspace_data/dataset_split/val' 写入验证集文件...
2025-09-13 21:50:51,906 - INFO - 所有文件已成功写入磁盘。
2025-09-13 21:50:52,976 - INFO - --- 开始验证拆分后的数据集 ---
2025-09-13 21:50:56,651 - INFO - ✅ PASSED: 训练集与验证集之间无样本ID交集 (无数据泄露)。
2025-09-13 21:50:56,652 - INFO - ✅ PASSED: 节点数与嵌入数在各子集中保持一致。
2025-09-13 21:50:57,073 - INFO - ✅ PASSED: 边表的引用完整性在各子集中得到保持。
2025-09-13 21:50:57,075 - INFO - 🎉 全部验证成功！数据集已准备就绪。
